# SEMESO 2023 - Mini Curso - Data Science no Mundo das Criptomoedas: da Blockchain ao MLOps 

Felipe Lana Machado

## Etapas:

1. Obtenção dos dados da moeda Illuvium (ILV) na rede ethereum.
    - Utilização da lib web3.py.
    - Utilização da API Alchemy. 
2. Pre-processamento dos dados obtidos.
3. Desenvolvimento de features dentro dos dados obtidos.
4. CI/CD com github actions.
5. Princípios do MLOps.

In [1]:
import warnings
import json
import os
from typing import List, Dict
from urllib3.exceptions import NotOpenSSLWarning
warnings.simplefilter('ignore', NotOpenSSLWarning)
from src.blockchain.get_ethereum_data import Blockchain
from src.utils.project_paths import DATA_RAW

/Users/krids/Documents/SEMESO23/Project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Obtenção dos dados

In [1]:
blockchain = Blockchain(abi='illuvium')
illuvium_started = 12084123
data = blockchain._get_events(start_block=illuvium_started)

list_of_files = os.listdir(DATA_RAW)
size = len([file for file in list_of_files if f'transfer_data_' in file])
with open(os.path.join(DATA_RAW, f'transfer_data_{size}.json'), 'w') as json_file:
    json.dump(data, json_file, indent=4)

/Users/krids/Documents/SEMESO23/Project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
100%|██████████| 613/613 [09:38<00:00,  1.06it/s]


## Pre-Processamento

In [2]:
def remove_duplicates_by_field(field: str, data: List[Dict]) -> List[Dict]:
    
    seen_ids = set()
    data = [seen_ids.add(x['txhash']) or x for x in data if x['txhash'] not in seen_ids]
    return data

In [4]:
len(data)

502130

In [9]:
x.drop_duplicates('txhash', inplace=True)

In [7]:
x['value_ilv'] = x.apply(lambda x: blockchain.w3.from_wei(x['value'],'ether'), axis=1)

In [11]:
x.reset_index(inplace=True, drop=True)

In [14]:
x.drop(columns=['index'], inplace=True)

In [18]:
x[x['from'] == '0x0000000000000000000000000000000000000000']

,txhash,blockNumber,from,to,value,value_ilv
0,0xa5b7f915aa6cd5789a79d2905540a08490bc215ac931...,12084124,0x0000000000000000000000000000000000000000,0x0691f5804d4227925F19b031821b530b48FFf38f,7000000000000000000000000,7000000
230264,0x45032394ca2f137288fed885a35b960af32df046eaa0...,15055460,0x0000000000000000000000000000000000000000,0x60CEEf10f9dd4a5d7874f22F461048eA96f475f6,32455825581532263104,32.455825581532263104
230410,0x84c6f769b93fb40ed7320459586b724052b71374e8de...,15056591,0x0000000000000000000000000000000000000000,0x7834626d2AA32e1Ca4EAA8cd481fd0BB3Eba064C,3907588938675326861,3.907588938675326861
230460,0xc32673df2dcf7e3f90f5a2414c72eb8d6c6936faba2b...,15056855,0x0000000000000000000000000000000000000000,0x0597433eD2970fef8976E106e77dB2E996F5A353,13689881826343393139,13.689881826343393139
230532,0x5c86d555d60190d38f111cddbdc051fd71eca4e5eb70...,15057420,0x0000000000000000000000000000000000000000,0x532D562bd356E8Bf71099eE074AF2737b0f452b0,61282521157777911981,61.282521157777911981
...,...,...,...,...,...,...
502259,0xf5ea47c9b15d272b75a52d8a538ea3cd0d60c9c2c373...,18212537,0x0000000000000000000000000000000000000000,0x1198C5E8cfF7c84a0BF77f988d1a71Ec1d2E5D24,3245881090118532864,3.245881090118532864
502260,0xac120eade57bba001d61ef32697906d49e1d883e8f08...,18212551,0x0000000000000000000000000000000000000000,0x1198C5E8cfF7c84a0BF77f988d1a71Ec1d2E5D24,2041221675056725572,2.041221675056725572
502270,0x2eb998f2078613a6b1497ef65f9647913b73c37700a9...,18212763,0x0000000000000000000000000000000000000000,0xAad1fDA8Ab6e75EaD08158E42D6883082E94bf45,2018544716524560075,2.018544716524560075
502273,0xee9d450deb03240502ba5b8efbe86950b8e7b3df1d55...,18212799,0x0000000000000000000000000000000000000000,0x204195e92904e02cB87409B2b0e1B7E6dDCfF07F,3682140676021400556,3.682140676021400556


## Desenvolvimento de Features

## CI / CD - Github Actions

## MLOPs